In [1]:
import os
import pandas as pd 
import polars as pl # like pandas, but much faster
import polars.selectors as cs
import numpy as np
import os, shutil, glob
from random import randint
import re, math
import datetime
import gc
from pathlib import Path

os.getcwd()


'/scratch2-shared/david/colopaint3D/notebooks'

### Read data

In [2]:
# pd.set_option("max_colwidth", 2000)
# pd.set_option("display.max_columns", 1000)
# pd.set_option("display.max_rows", 1000)

# dirlist = ['/share/data/cellprofiler/automation/results/spher010-DLD1-L5/3958/5244/'
#            #, '/share/data/cellprofiler/automation/results/spher010-HCT116-L6/3943/5241/'
#            , '/share/data/cellprofiler/automation/results/spher010-HCT15-L2/3954/5246/'
#            , '/share/data/cellprofiler/automation/results/spher010-HT29-L4/3962/5247/']
# # BaseDir = '/share/data/analyses/christa/colopaint3D/pilots1-10/results_spheroid'
rootDir = '/share/data/cellprofiler/automation/results'
# BaseImgDir = '/share/mikro2/nikon/spheroid-test/pilot12-HCT116-day2/'
# BaseDir = '/share/data/cellprofiler/automation/results/pilot12-HCT116-day2/3797/5112'
# sorted(os.listdir(rootDir))

### Input directories

In [3]:
OutputDir = 'output/spher_colo52_v1/1_FeaturesImages'
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)
NameContains = ''
InputFolders = pl.read_csv('./settings/filemap.csv')
print(InputFolders )
            


shape: (3, 1)
┌───────────────────┐
│ feature_names     │
│ ---               │
│ str               │
╞═══════════════════╡
│ featICF_nuclei    │
│ featICF_cells     │
│ featICF_cytoplasm │
└───────────────────┘


In [4]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2024-03-13 16:24:53


In [5]:
def print_time(msg=None):
    now = datetime.datetime.now()
    print(now.strftime('%Y-%m-%d %H:%M:%S'),msg or "")
float_columns=[pl.col(pl.Float32),pl.col(pl.Float64)]

In [6]:
metaEx = pl.read_csv('/home/jovyan/scratch2-shared/david/colopaint3D/notebooks/settings/spher-colo52-v1-import-inputFiles-and-PLAIDresults.csv')
barcodes = metaEx.select(pl.col('barcode')).unique()
barcodes = barcodes['barcode']
barcodes.to_list()
dirlist = [f'{rootDir}/{barcode}' for barcode in barcodes]
# mdf = pd.read_csv('/home/jovyan/scratch2-shared/david/colopaint3D/notebooks/settings/spher-colo52-v1-import-inputFiles-and-PLAIDresults.csv', sep=',')

# barcodes = mdf['barcode'].unique()

# metaEx = metaEx.filter(pl.col('cmpdname')=='dmso')
# meta1 = metaEx.select(['barcode','well_id','cmpdname'])
# meta2 = metaEx.select(['barcode','well_id','pathway', 'cell_line'])
# meta3 = meta1.join(meta2, left_on=['barcode', 'well_id'], right_on=['barcode', 'well_id'])

In [7]:
metaEx.head()


index,layout_id,well_id,image_id,cp_id,barcode,plate_well,cmpd_code,cmpdname,solvent,cmpd_conc,cmpd_conc_unit,stock_conc,stock_conc_unit,cmpd_vol,cmpd_vol_unit,well_vol,well_vol_unit,pert_type,article_id,target,pathway,pubchemID,smiles,inkey,target_type,clinical_status,cell_line
i64,str,str,i64,i64,str,str,str,str,str,f64,str,f64,str,f64,str,f64,str,str,str,str,str,f64,str,str,str,str,str
0,"""spher-colo52-v…","""B02""",4185,5532,"""PB000137""","""PB000137_B02""","""colo-006""","""PD0325901""","""dmso""",3.0,"""µM""",10.0,"""mM""",30.0,"""nl""",40.0,"""µl""","""trt""","""S1036""","""MEK""","""MAPK""",9.826528e6,"""OCC(O)CONC(=O)…","""SUDAHWBOROXANE…","""Targeted""","""Phase 2""","""HCT116"""
1,"""spher-colo52-v…","""B03""",4185,5532,"""PB000137""","""PB000137_B03""","""colo-018""","""Paclitaxel""","""dmso""",0.1,"""µM""",0.1,"""mM""",100.0,"""nl""",40.0,"""µl""","""trt""","""S1150""","""Autophagy,Micr…","""Cytoskeletal S…",441276.0,"""CC(=O)OC1C(=O)…","""RCINICONZNJXQF…","""Cytotoxic""","""Preclinical""","""HCT116"""
2,"""spher-colo52-v…","""B04""",4185,5532,"""PB000137""","""PB000137_B04""","""colo-009""","""Olaparib (AZD2…","""dmso""",3.0,"""µM""",10.0,"""mM""",30.0,"""nl""",40.0,"""µl""","""trt""","""S1060""","""PARP""","""DNA Damage""",2.3725625e7,"""FC1=C(C=C(CC2=…","""FDLYAMZZIXQODN…","""Targeted""","""Launched""","""HCT116"""
3,"""spher-colo52-v…","""B05""",4185,5532,"""PB000137""","""PB000137_B05""","""colo-012""","""SB216763""","""dmso""",10.0,"""µM""",10.0,"""mM""",100.0,"""nl""",40.0,"""µl""","""trt""","""S1075""","""GSK-3""","""PI3K/Akt/mTOR""",176158.0,"""C[N]1C=C(C2=C1…","""JCSGFHVFHSKIJH…","""Targeted""","""Preclinical""","""HCT116"""
4,"""spher-colo52-v…","""B06""",4185,5532,"""PB000137""","""PB000137_B06""","""colo-008""","""Vorinostat (SA…","""dmso""",3.0,"""µM""",10.0,"""mM""",30.0,"""nl""",40.0,"""µl""","""trt""","""S1047""","""Autophagy,HDAC…","""Epigenetics""",5311.0,"""ONC(=O)CCCCCCC…","""WAEXFXRVDQXREF…","""Targeted""","""Launched""","""HCT116"""


In [8]:
# dirlist[0]
# dirlist[0].split('/')

In [9]:
BaseDir = dirlist[0]
mdf_op = metaEx.filter(pl.col('barcode') == BaseDir.split('/')[-1])
image_ids = mdf_op.select(pl.col('image_id')).unique().to_series().to_list()[-1]
cp_ids = mdf_op.select(pl.col('cp_id')).unique().to_series().to_list()[-1]
df = pl.read_parquet(f'{BaseDir}/{image_ids}/{cp_ids}/featICF_nuclei.parquet')

In [10]:
median = df.select(float_columns).median()
# df4 = df.select(float_columns)
mad = pl.concat([df.select((pl.col(c)-pl.col(c).median()).abs().median()) for c in df.select(float_columns).columns], how='horizontal')
# mad = pl.concat(mad, how='horizontal')

In [11]:
# mad = df.select((pl.col('AreaShape_Extent')-pl.col('AreaShape_Extent').median()).abs().median())
mad

AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_Extent,AreaShape_FormFactor,AreaShape_MajorAxisLength,AreaShape_MaxFeretDiameter,AreaShape_MaximumRadius,AreaShape_MeanRadius,AreaShape_MedianRadius,AreaShape_MinFeretDiameter,AreaShape_MinorAxisLength,AreaShape_Orientation,AreaShape_Perimeter,AreaShape_Solidity,AreaShape_Zernike_0_0,AreaShape_Zernike_1_1,AreaShape_Zernike_2_0,AreaShape_Zernike_2_2,AreaShape_Zernike_3_1,AreaShape_Zernike_3_3,AreaShape_Zernike_4_0,AreaShape_Zernike_4_2,AreaShape_Zernike_4_4,AreaShape_Zernike_5_1,AreaShape_Zernike_5_3,AreaShape_Zernike_5_5,AreaShape_Zernike_6_0,AreaShape_Zernike_6_2,AreaShape_Zernike_6_4,AreaShape_Zernike_6_6,AreaShape_Zernike_7_1,AreaShape_Zernike_7_3,AreaShape_Zernike_7_5,AreaShape_Zernike_7_7,…,RadialDistribution_ZernikePhase_PHAandWGA_8_6,RadialDistribution_ZernikePhase_PHAandWGA_8_8,RadialDistribution_ZernikePhase_PHAandWGA_9_1,RadialDistribution_ZernikePhase_PHAandWGA_9_3,RadialDistribution_ZernikePhase_PHAandWGA_9_5,RadialDistribution_ZernikePhase_PHAandWGA_9_7,RadialDistribution_ZernikePhase_PHAandWGA_9_9,RadialDistribution_ZernikePhase_SYTO_0_0,RadialDistribution_ZernikePhase_SYTO_1_1,RadialDistribution_ZernikePhase_SYTO_2_0,RadialDistribution_ZernikePhase_SYTO_2_2,RadialDistribution_ZernikePhase_SYTO_3_1,RadialDistribution_ZernikePhase_SYTO_3_3,RadialDistribution_ZernikePhase_SYTO_4_0,RadialDistribution_ZernikePhase_SYTO_4_2,RadialDistribution_ZernikePhase_SYTO_4_4,RadialDistribution_ZernikePhase_SYTO_5_1,RadialDistribution_ZernikePhase_SYTO_5_3,RadialDistribution_ZernikePhase_SYTO_5_5,RadialDistribution_ZernikePhase_SYTO_6_0,RadialDistribution_ZernikePhase_SYTO_6_2,RadialDistribution_ZernikePhase_SYTO_6_4,RadialDistribution_ZernikePhase_SYTO_6_6,RadialDistribution_ZernikePhase_SYTO_7_1,RadialDistribution_ZernikePhase_SYTO_7_3,RadialDistribution_ZernikePhase_SYTO_7_5,RadialDistribution_ZernikePhase_SYTO_7_7,RadialDistribution_ZernikePhase_SYTO_8_0,RadialDistribution_ZernikePhase_SYTO_8_2,RadialDistribution_ZernikePhase_SYTO_8_4,RadialDistribution_ZernikePhase_SYTO_8_6,RadialDistribution_ZernikePhase_SYTO_8_8,RadialDistribution_ZernikePhase_SYTO_9_1,RadialDistribution_ZernikePhase_SYTO_9_3,RadialDistribution_ZernikePhase_SYTO_9_5,RadialDistribution_ZernikePhase_SYTO_9_7,RadialDistribution_ZernikePhase_SYTO_9_9
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
140.377869,146.693787,0.057694,0.099988,5.278965,0.035528,0.042691,7.707695,7.521484,2.394749,0.798674,0.697598,5.07041,5.010363,43.956638,19.485291,0.007212,0.087572,0.018469,0.019057,0.015927,0.006482,0.006392,0.01629,0.007433,0.006987,0.005543,0.004065,0.002884,0.005751,0.005626,0.003529,0.003289,0.00332,0.002913,0.002362,0.001572,…,1.56037,1.56331,1.583952,1.563431,1.574934,1.560026,1.57781,0.0,1.557205,0.0,1.549706,1.57219,1.546507,0.0,1.575588,1.552418,1.595531,1.589931,1.558367,0.0,1.588884,1.564042,1.558731,1.585542,1.562456,1.563331,1.6173,0.0,1.57878,1.579449,1.56165,1.56366,1.584494,1.563835,1.575225,1.55959,1.577095


In [12]:
# for BaseDir in dirlist:
#     mdf_op = metaEx.filter(pl.col('barcode') == BaseDir.split('/')[-1])
#     image_id = mdf_op.select(pl.col('image_id')).unique().to_series().to_list()[-1]
#     cp_id = mdf_op.select(pl.col('cp_id')).unique().to_series().to_list()[-1]
#     print(f'{BaseDir}/{image_id}/{cp_id}')

In [13]:
use_clipping = False

std_mean = True

In [14]:
# medianImageAllPlates = pd.DataFrame()
# meanImageAllPlates = pd.DataFrame()
# wellmismatch = 0
# df_scall = pd.DataFrame()
# df_meanall = pd.DataFrame()
# df_medianall = pd.DataFrame()
# DataFrameDictionary = {}s
ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']
# metaEx = pl.read_csv('/home/jovyan/scratch2-shared/david/colopaint3D/notebooks/settings/spher010-metadata.csv')
for BaseDir in dirlist:
    mdf_op = metaEx.filter(pl.col('barcode') == BaseDir.split('/')[-1])
    image_id = mdf_op.select(pl.col('image_id')).unique().to_series().to_list()[-1]
    cp_id = mdf_op.select(pl.col('cp_id')).unique().to_series().to_list()[-1]
    print(f'{BaseDir}/{image_id}/{cp_id}')
    BaseDir = f'{BaseDir}/{image_id}/{cp_id}'


    # nuclei = pl.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet').add_prefix('Nuclei_').reset_index()
    # cytoplasm = pl.read_parquet(BaseDir+f'/{ObjectList[1]}.parquet').add_prefix('Cytoplasm_').reset_index()
    # cells = pl.read_parquet(BaseDir+f'/{ObjectList[2]}.parquet').add_prefix('Cells_').reset_index()
    # f_df=pl.read_parquet(f)
    # f_df=f_df.rename({x:f'{feature_set_name}_{x}' for x in f_df.columns})
    

    nuclei = pl.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet')
    nuclei=nuclei.rename({x:f'Nuclei_{x}' for x in nuclei.columns})
    cytoplasm = pl.read_parquet(BaseDir+f'/{ObjectList[1]}.parquet')
    cytoplasm=cytoplasm.rename({x:f'Cytoplasm_{x}' for x in cytoplasm.columns})
    cells = pl.read_parquet(BaseDir+f'/{ObjectList[2]}.parquet')
    cells=cells.rename({x:f'Cells_{x}' for x in cells.columns})
    # step 1: Take the mean values of 'multiple nuclei' belonging to one cell

    nuclei = nuclei.group_by([
        "Nuclei_Metadata_Barcode","Nuclei_Metadata_Well",
        "Nuclei_Parent_cells", 'Nuclei_Metadata_Site'
    ]).mean()
    
    df = cytoplasm.join(nuclei,
                how='left', 
                right_on=['Nuclei_Metadata_Well', 'Nuclei_Metadata_Site', 'Nuclei_Parent_cells', 'Nuclei_Metadata_Barcode'],
                left_on = ['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site', 'Cytoplasm_ObjectNumber', 'Cytoplasm_Metadata_Barcode'])
                
    df = df.join(cells, how='left', 
                    left_on=['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site','Cytoplasm_ObjectNumber', 'Cytoplasm_Metadata_Barcode'],
                    right_on = ['Cells_Metadata_Well','Cells_Metadata_Site',"Cells_ObjectNumber", 'Cells_Metadata_Barcode'])

    # print_time("initial merging")
    print('part1')
    
    # deduplicate barcode/well/site - renamed cytoplasm_Metadata* to Metadata* and removes nuclei_* etc
    unique_metadata_feature_names=['Metadata_Barcode','Metadata_Well','Metadata_Site']
    df=df.rename({f'Cytoplasm_{suffix}':suffix for suffix in unique_metadata_feature_names})
                
    # for some reason, the site is parsed as float, even though it really should be an int
    if df['Metadata_Site'].dtype in [np.dtype('float32'), np.dtype('float64')]:
        # sometimes, for some reason, site indices are inf/nan
        site_is_nan_mask=np.isnan(df['Metadata_Site'])
        site_is_inf_mask=np.isinf(df['Metadata_Site'])
        
        try:
            num_sites_nan=np.sum(site_is_nan_mask)
            num_sites_inf=np.sum(site_is_inf_mask)
            assert num_sites_nan==0, f"found nan site values (n = {num_sites_nan})"
            assert num_sites_inf==0, f"found inf site values (n = {num_sites_inf})"
        except AssertionError as e:
            print(f"info - this issue was automatically circumvented in the code : {e}")
            df=df[~(site_is_inf_mask|site_is_nan_mask)]
            
        num_metadata_site_entries_nonint=np.sum(np.abs(df['Metadata_Site']%1.0)>1e-6)
        assert num_metadata_site_entries_nonint==0, f"ERROR : {num_metadata_site_entries_nonint} imaging sites don't have integer indices. that should not be the case, and likely indicates a bug."
        
        #Should use np.round, no? TODO ask patrick. Truncation Errors are annoying.
        df['Metadata_Site'] = df['Metadata_Site'].astype(np.dtype('int32'))

    #Adding Compound Metadata to each row
    df = df.join(mdf_op.rename({x:f"Metadata_cmpd_{x}" for x in mdf_op.columns}),left_on='Metadata_Well',right_on='Metadata_cmpd_well_id')    
    print("processed some metadata")
    
    # drop all rows that contain nan
    num_rows_before_nan_trim = df.shape[0]
    for col in df.select([pl.col(pl.Float32),pl.col(pl.Float64)]).columns:
        before_drop=df.shape[0]
        df=df.filter(pl.col(col).is_not_null())
        after_drop=df.shape[0]

        #num_values_dropped=before_drop-after_drop
        #if num_values_dropped>0:
        #    print(f"dropped {num_values_dropped} rows due to NaNs in column {col}")

    num_rows_after_nan_trim = df.shape[0]
    
    print_time("dropped NaNs")
    
    # Clip outliers
    if use_clipping:
        float_cols = [c for c_name,c_dtype in zip(df.columns,df.dtypes) if "float" in str(c_dtype)]
        lower_quantile = df.select(float_cols).quantile(0.01)
        upper_quantile = df.select(float_cols).quantile(0.99)
        print("calced quantiles")

        for col in float_cols:
            df = df.with_column(
                pl.col(col).clip(lower=lower_quantile[col],upper=upper_quantile[col])
            )
            
        print("clipped")
    
    # for some reason, DMSO is used as batch id for cells that are not treated with a drug
    df_DMSO=df.filter(pl.col('Metadata_cmpd_cmpdname')=='dmso')
    assert df_DMSO.shape[0]>0, "did not find any wells 'treated' with DMSO"
    
    # df_DMSO = wells_with_dmso
    # use join to quickly select the relevant rows
    # but add unique prefix to compound information columns (to avoid name collisions)
    # df_DMSO = df.join(wells_with_dmso.rename({x:f"Metadata_cmpd_{x}" for x in wells_with_dmso.columns}),left_on='Metadata_Well',right_on='Metadata_cmpd_well_id')
    # # then remove the compound information columns again
    # df_DMSO = df_DMSO.select([x for x in df_DMSO.columns if not x.startswith('compoundinfo_')])
    # assert df_DMSO.shape[0]>0, "something is very wrong"
    if std_mean:
        mu = df_DMSO.select(float_columns).mean()
        for col in mu.columns:
            if mu[col].is_null().any():
                raise RuntimeError(f"some mean value in column {col,i} is nan?!")
            if mu[col].is_infinite().any():
                raise RuntimeError(f"some mean value in column {col,i} is infinite?!")
                    
        std = df_DMSO.select(float_columns).std()
        # replace 0 with 1 (specifically not clip) to avoid div by zero
        std = std.select([pl.col(c).map_dict({0: 1}, default=pl.col(c)) for c in std.columns])
        
        for i,col in enumerate(std.columns):
            if std[col].is_null().any():
                raise RuntimeError(f"some std value in column {col,i} is nan?!")
            if std[col].is_infinite().any():
                raise RuntimeError(f"some std value in column {col,i} is infinite?!")
            if (std[col]==0).any():
                raise RuntimeError(f"unexpected 0 in column {col}")
        
        print_time("calculated DMSO distribution")

        df_standardized = df.with_columns([(pl.col(c) - mu[c]) / (std[c]+0.01) for c in mu.columns])
    
    else:
        median = df_DMSO.select(float_columns).median()

        # Check for null or infinite medians and raise errors if found
        for col in median.columns:
            if median[col].is_null().any():
                raise RuntimeError(f"some median value in column {col} is nan?!")
            if median[col].is_infinite().any():
                raise RuntimeError(f"some median value in column {col} is infinite?!")

        # mad = df_DMSO.select([(pl.col(c) - pl.col(c).median()).abs().alias(c) for c in float_columns])
        mad = pl.concat([df.select((pl.col(c)-pl.col(c).median()).abs().median()) for c in df.select(float_columns).columns], how='horizontal')
        # mad = df_DMSO.select([pl.abs(pl.col(c) - median[c]).median().alias(c) for c in float_columns])


        # Ensure MAD values are not zero to avoid division by zero
        # Here we replace 0 with a small value (e.g., 0.01) instead of 1 to maintain the scale of MAD
        # mad = mad.select([pl.when(pl.col(c) == 0).then(0.01).otherwise(pl.col(c)).alias(c) for c in mad.columns])
        mad = mad.select([pl.col(c).map_dict({0: 0.01}, default=pl.col(c)) for c in mad.columns])

        # Check for null, infinite, or unexpected zero values in MAD and raise errors if found
        for i, col in enumerate(mad.columns):
            if mad[col].is_null().any():
                raise RuntimeError(f"some MAD value in column {col,i} is nan?!")
            if mad[col].is_infinite().any():
                raise RuntimeError(f"some MAD value in column {col,i} is infinite?!")
            if (mad[col] == 0).any():
                raise RuntimeError(f"unexpected 0 in column {col}")

        print_time("calculated DMSO distribution")
        df_standardized = df.with_columns([(pl.col(c) - median[c]) / (mad[c]) for c in median.columns])

        


    #Checking for NaNs
    found_nan=False
    if std_mean:
        for i,col in enumerate(mu.columns):
            if df_standardized[col].is_null().any():
                found_nan=True
                print(f"some value in column {col,i} is nan")
        
        if found_nan:
            raise RuntimeError("found nan")
    else:
        for i,col in enumerate(median.columns):
            if df_standardized[col].is_null().any():
                found_nan=True
                print(f"some value in column {col,i} is nan")
        
        if found_nan:
            raise RuntimeError("found nan")
    
    # write back
    df=df.with_columns([df_standardized[c] for c in df_standardized.columns])
    


    print_time("normalized to DMSO distribution")

    # counts objects identified by segmentation, i.e. should be cells
    num_objects = df.shape[0]
    
    fraction_objects_containing_nan=1-(num_rows_after_nan_trim/num_rows_before_nan_trim)
    print_time(f"num objects (cells) {num_objects} ({(fraction_objects_containing_nan*100):.2f}% were NaN)")
    
    # Save Single Cell Parquets to outputs
    ScOut = f'{OutputDir}/SingleCell/'
    if not os.path.exists(ScOut): 
        os.makedirs(ScOut)
    df.write_parquet(f'{ScOut}/{df.select(pl.col("Metadata_Barcode")).unique().item()}.parquet')

    # # group/combine by well
    df_agg=df.drop(columns=['Metadata_Site'])
    df_float_columns=set(list(df_agg.select(float_columns).columns))
    group_by_columns=['Metadata_Barcode','Metadata_Well']
    other_columns=set(list(df_agg.columns))-df_float_columns-set(group_by_columns)
    # # group by mean for all float features, and group by first for all non-float columns (indices and string metadata)
    group_by_aggregates=[
        *[pl.mean(x) for x in list(df_float_columns)],
        *[pl.first(x) for x in list(other_columns)]
    ]
    df_agg_mean=df_agg.group_by(group_by_columns).agg(group_by_aggregates)
    MeanOut = f'{OutputDir}/MeanCell/'
    if not os.path.exists(MeanOut): 
        os.makedirs(MeanOut)
    df_agg_mean.write_parquet(f'{MeanOut}/{df.select(pl.col("Metadata_Barcode")).unique().item()}.parquet')
    print_time("binned mean data per well")

    df_float_columns=set(list(df_agg.select(float_columns).columns))
    group_by_columns=['Metadata_Barcode','Metadata_Well']
    other_columns=set(list(df_agg.columns))-df_float_columns-set(group_by_columns)
    group_by_aggregates=[
        *[pl.median(x) for x in list(df_float_columns)],
        *[pl.first(x) for x in list(other_columns)]
    ]
    df_agg_median=df_agg.group_by(group_by_columns).agg(group_by_aggregates)
    MedianOut = f'{OutputDir}/MedianCell/'
    if not os.path.exists(MedianOut): 
        os.makedirs(MedianOut)
    df_agg_median.write_parquet(f'{MedianOut}/{df.select(pl.col("Metadata_Barcode")).unique().item()}.parquet')
    print_time("binned median data per well")
    
    # # add compound information    
    # combined_per_well=combined_per_well.join(compound_layout,how='left',left_on=["Metadata_Well"],right_on=["compound_well_id"])
    # print_time("added compound information")
    
    # cpi = CellprofilerImage(
    #     image_plate_dir=timepoint_info.image_plate_dir,
    #     image_id=cp_out_path,
    #     pipeline_id_qc=str(pipeline_id_qc),
    #     pipeline_id_features=str(pipeline_id_features),
    #     # "-1" because timepoints in the metadata file are 1-indexed, but 0-indexed in the filesystem
    #     timepoint=str(int(cellprofiler_image_timepoints[i_i])-1),
    #     combined_per_well=combined_per_well
    # )
    
    # display(combined_per_well.head(2))
    
    # print_time(f"timepoint {Path(cpi.image_plate_dir).name} done")
    
    # cellprofiler_images.append(cpi)
# df_scall.to_parquet('{}/spheroidSingleCell.parquet'.format(OutputDir))
# df_meanall.to_parquet('{}/spheroidMean.parquet'.format(OutputDir))
# df_medianall.to_parquet('{}/spheroidMedian.parquet'.format(OutputDir))

/share/data/cellprofiler/automation/results/PB000141/4187/5546
part1
processed some metadata
2024-03-13 16:25:42 dropped NaNs
2024-03-13 16:25:47 calculated DMSO distribution
2024-03-13 16:25:49 normalized to DMSO distribution
2024-03-13 16:25:49 num objects (cells) 230567 (0.03% were NaN)
2024-03-13 16:26:26 binned mean data per well
2024-03-13 16:26:28 binned median data per well
/share/data/cellprofiler/automation/results/PB000137/4185/5532
part1
processed some metadata
2024-03-13 16:27:10 dropped NaNs
2024-03-13 16:27:14 calculated DMSO distribution
2024-03-13 16:27:16 normalized to DMSO distribution
2024-03-13 16:27:16 num objects (cells) 384940 (0.02% were NaN)
2024-03-13 16:28:20 binned mean data per well
2024-03-13 16:28:23 binned median data per well
/share/data/cellprofiler/automation/results/PB000138/4188/5547
part1
processed some metadata
2024-03-13 16:29:05 dropped NaNs
2024-03-13 16:29:10 calculated DMSO distribution
2024-03-13 16:29:12 normalized to DMSO distribution
202

In [15]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2024-03-13 16:33:45


In [16]:
# df_scall.head()

In [17]:
# df_meanall.head()

In [18]:
# toNpy = df_scall.loc[:,~df_scall.columns.str.contains('Metadata_')]
# toNpy.reset_index(inplace=True, drop=True)
# toNpy = toNpy.copy()
# dataNpy = toNpy.to_numpy()
# np.isnan(dataNpy).any()
# colnames = toNpy.columns